In [122]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/v21
bak
model_gbdt_v21.ipynb
model_lgb_v21.ipynb
process_v21.ipynb
train.dat.v21


In [123]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
import numpy as np
import time 
from sklearn.preprocessing import OrdinalEncoder

pd.set_option('display.max_columns',100)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'

print(suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

v21
process time :  2025-08-31 17:15:11


In [124]:
mode="test"

NROWS = 10000000000000000

if mode == "train":
    base_input_path = train_path
    tx_input_path = train_tx_path
    output_path = "train.dat.%s" % suffix
else:
    base_input_path = test_path
    tx_input_path = test_tx_path
    output_path = "test.dat.%s" % suffix

print(mode, base_input_path, tx_input_path, output_path)

test ../../testaa.csv ../../testaa_bank_statement.csv test.dat.v21


#  基础特征

In [125]:
!pwd
df = pd.read_csv(base_input_path, index_col=['id'], nrows=NROWS)
# print(df.head())
print(df.shape)

df['balance_account_avg'] = df['balance'] / df['balance_accounts']
df['loan_term_avg'] = df['loan'] / df['term']
df['balance_accounts_ratio'] = df['balance_accounts'] / df['total_accounts']

#log 处理
df['loan_log'] = df['loan'].apply(lambda x : np.log(x))
df['balance_account_avg_log'] = df['balance_account_avg'].apply(lambda x : np.log(x))
df['loan_term_avg_log'] = df['loan_term_avg'].apply(lambda x : np.log(x))
df['balance_accounts_ratio_log'] = df['balance_accounts_ratio'].apply(lambda x : np.log(x))
df['interest_rate_log'] = df['interest_rate'].apply(lambda x : np.log(x))

df['balance_log'] = df['balance'].apply(lambda x : np.log(x))
df['balance_limit_log'] = df['balance_limit'].apply(lambda x : np.log(x))
df['balance_accounts_log'] = df['balance_accounts'].apply(lambda x : np.log(x))

# zip code
df['zip_province']  = df['zip_code'].apply(lambda x : str(x)[:2])
df['zip_city']      = df['zip_code'].apply(lambda x : str(x)[:4])

#level 处理
df['level_hash'] = df['level'].apply(lambda x : hash(x) % 1000 )
encoder = OrdinalEncoder(categories=[['A0','A1','A2','A3','A4','A5','B0','B1','B2','B3','B4','B5','C0','C1','C2','C3','C4','C5','D0','D1','D2','D3','D4','D5','E0','E1','E2','E3','E4','E5']]) 
df['level_ord'] = encoder.fit_transform(df[['level']].values)

df['grade'] = df['level'].apply( lambda x : str(x)[0])  #提取ABCDE
df['grade'] = df['grade'].map({'A':1 , 'B':2, 'C':3, 'D':4, 'E':5})


# 分桶处理
bins_num = 50
tmp_labels = ['%d' % i for i in range(bins_num)]
df['interest_rate_cut'] = pd.cut(df['interest_rate'], bins=bins_num, labels = tmp_labels)

tmp_labels = ['%d' % i for i in range(bins_num)]
df['interest_rate_log_cut'] = pd.cut(df['interest_rate_log'], bins=bins_num, labels = tmp_labels)

# 等频
tmp_labels = ['%d' % i for i in range(bins_num)]
df['balance_cut'] = pd.qcut(df['balance'], q=bins_num, labels = tmp_labels, duplicates='drop')

df['loan_cut'] = pd.qcut(df['loan'], q=bins_num,  duplicates='drop')
loan_labels = np.unique(df['loan_cut'].values.codes).tolist()
df['loan_cut'] = pd.qcut(df['loan'], q=bins_num,  labels = loan_labels,  duplicates='drop')

tmp_labels = ['%d' % i for i in range(bins_num)]
df['balance_limit_cut'] = pd.qcut(df['balance_limit'], q=bins_num, labels = tmp_labels, duplicates='drop')

df['loan_term_avg_cut'] = pd.qcut(df['loan_term_avg'], q=bins_num,  duplicates='drop')
loan_labels = np.unique(df['loan_term_avg_cut'].values.codes).tolist()
df['loan_term_avg_cut'] = pd.qcut(df['loan_term_avg'], q=bins_num,  labels = loan_labels,  duplicates='drop')

tmp_labels = ['%d' % i for i in range(bins_num)]
df['balance_account_avg_cut'] = pd.qcut(df['balance_account_avg'], q=bins_num, labels = tmp_labels, duplicates='drop')

# 时间处理
df['record_time_format'] = df['record_time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
df['record_time_year'] = df['record_time_format'].map(lambda x : x.year)
df['record_time_month'] = df['record_time_format'].map(lambda x : x.month)      # 探索周期性
df['record_time_week'] = df['record_time_format'].map(lambda x : x.week)      # 探索周期性
df['record_time_year_month'] = df['record_time_format'].map(lambda x : x.strftime('%Y%m'))

df.drop(['record_time_format'], axis=1, inplace=True)
df

/d/GH/GitWorkSpace/bank_model_competiton/data/v21
(20054, 17)


,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,balance_account_avg,loan_term_avg,balance_accounts_ratio,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,zip_province,zip_city,level_hash,level_ord,grade,interest_rate_cut,interest_rate_log_cut,balance_cut,loan_cut,balance_limit_cut,loan_term_avg_cut,balance_account_avg_cut,record_time_year,record_time_month,record_time_week,record_time_year_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,0,1054600080,795400642,17,9,10100.0,8233.00,B0,914.777778,165.277778,0.529412,8.691146,6.818681,5.107628,-0.635989,2.646175,9.015906,9.220291,2.197225,51,5120,54,6.0,2,17,27,34,2,28,3,32,2003,6,23,200306
53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,0,1172882338,756003297,17,9,58600.0,23460.00,A0,2606.666667,426.388889,0.529412,9.638871,7.865828,6.055352,-0.635989,1.658228,10.063052,10.978490,2.197225,40,4018,212,0.0,1,0,0,46,15,47,14,47,2007,3,9,200703
53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,0,1277511093,658715225,17,9,15500.0,7853.00,B1,872.555556,306.944444,0.529412,9.310186,6.771426,5.726667,-0.635989,2.613740,8.968651,9.648595,2.197225,32,3216,868,7.0,2,16,26,34,12,32,11,30,2010,6,25,201006
53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,0,1198371199,774316843,17,9,10100.0,5815.00,B2,646.111111,426.388889,0.529412,9.638871,6.470971,6.055352,-0.635989,2.653242,8.668196,9.220291,2.197225,23,2313,222,8.0,2,17,27,31,15,28,14,15,2007,12,51,200712
53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,1,1214957804,832207268,17,9,13100.0,8225.00,A3,913.888889,216.666667,0.529412,8.961879,6.817709,5.378360,-0.635989,2.302585,9.014934,9.480368,2.197225,17,1711,696,3.0,1,9,18,34,5,31,7,32,2008,7,27,200807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,0,1125964800,1018224000,6,3,3818.0,2224.69,A4,741.563333,833.333333,0.500000,9.210340,6.608761,6.725434,-0.693147,2.936513,7.707373,8.247482,1.098612,60,6011,625,4.0,1,26,35,20,9,21,23,20,2005,9,36,200509
73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,0,1157068800,1054425600,6,6,5502.0,4126.71,B4,687.785000,833.333333,1.000000,9.210340,6.533476,6.725434,0.000000,3.377588,8.325236,8.612867,1.791759,60,6011,403,10.0,2,46,48,28,9,24,23,17,2006,9,35,200609
73531,0,4.0,601408,1,11000,12,24.75,1144108800,0,0,1111622400,1037404800,8,3,4844.0,2710.96,A3,903.653333,916.666667,0.375000,9.305651,6.806446,6.820744,-0.980829,3.208825,7.905058,8.485496,1.098612,60,6014,696,3.0,1,37,43,23,11,23,24,32,2005,3,12,200503


#  交易特征处理

In [126]:
df_tx = pd.read_csv(tx_input_path, nrows = NROWS, index_col=['id'])
df_tx['time_format'] = df_tx['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
print(df_tx)

#days_diff, tm_count, total_amount, amount_1, amount_0, total_amount_day_avg, amount_1_day_avg, amount_0_day_avg
print(df_tx.shape)

df_tx_stat = pd.DataFrame()
df_tx_stat['tx_time_max'] = df_tx.groupby('id')['time_format'].agg('max')
df_tx_stat['tx_time_min'] = df_tx.groupby('id')['time_format'].agg('min')

df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat['tx_count'] = df_tx.groupby('id')['amount'].agg('count')
df_tx_stat['total_amount'] = df_tx.groupby('id')['amount'].agg('sum')

df_tx_stat['1_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 1]['amount'].sum())
df_tx_stat['0_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 0]['amount'].sum())

df_tx_stat['total_amount_avg']  = df_tx_stat['total_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['1_amount_avg']  = df_tx_stat['1_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['0_amount_avg']  = df_tx_stat['0_amount'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg2']  = df_tx_stat['total_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['1_amount_avg2']  = df_tx_stat['1_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['0_amount_avg2']  = df_tx_stat['0_amount'] / df_tx_stat['tx_count'] 

#交易活跃度： 效果次数/天数
df_tx_stat['tx_count_avg']  = df_tx_stat['tx_count'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['tx_tmstp_max'] = df_tx.groupby('id')['time'].agg('max')
df_tx_stat['tx_tmstp_min'] = df_tx.groupby('id')['time'].agg('min')
df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)
df_tx_stat

             time  direction       amount         time_format
id                                                           
56693  1144195200          1   542.210000 2006-04-05 08:00:00
56693  1145145600          1   283.490000 2006-04-16 08:00:00
56693  1145145600          0  5604.080000 2006-04-16 08:00:00
56693  1146787200          1   539.380000 2006-05-05 08:00:00
56693  1147737600          0  5951.750000 2006-05-16 08:00:00
...           ...        ...          ...                 ...
71870  1160956800          1   493.403945 2006-10-16 08:00:00
71870  1161043200          0     9.462382 2006-10-17 08:00:00
71870  1161388800          1   222.936072 2006-10-21 08:00:00
71870  1161475200          0   222.936072 2006-10-22 08:00:00
71870  1161475200          1    20.908367 2006-10-22 08:00:00

[646666 rows x 4 columns]
(646666, 4)


,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min
id,,,,,,,,,,,,,,
53480,165,46,61815.520000,11245.520000,50570.000000,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826,0.278788,1053302400,1039046400
53481,144,56,38555.840000,20161.940000,18393.900000,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500,0.388889,1170806400,1158364800
53482,172,74,81589.800000,44977.460000,36612.340000,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351,0.430233,1275350400,1260489600
53483,159,73,48205.320000,24048.520000,24156.800000,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068,0.459119,1195776000,1182038400
53484,174,91,78113.610000,29797.860000,48315.750000,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308,0.522989,1214438400,1199404800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73515,94,25,9477.597620,5109.004148,4368.593472,100.825507,54.351108,46.474399,379.103905,204.360166,174.743739,0.265957,1161475200,1153353600
73517,180,272,212302.562407,126112.342194,86190.220212,1179.458680,700.624123,478.834557,780.524126,463.648317,316.875810,1.511111,1161475200,1145923200
73522,180,116,45701.276358,24957.995111,20743.281247,253.895980,138.655528,115.240451,393.976520,215.155130,178.821390,0.644444,1161475200,1145923200


# 合并特征、处理缺失值、保存结果

In [127]:
# 合并交易特征
df_concat = pd.concat([df, df_tx_stat], axis=1)
print(df_concat.shape)
df_concat.reset_index(inplace=True)
df_concat = df_concat.rename(columns={'index':'id'})
print(df_concat.columns)

# 缺失值处理
df_concat = df_concat.replace([np.inf, -np.inf], np.nan)

col_str = 'career,balance_limit_cut'
for col_name in col_str.split(','):
    value = df_concat[col_name].mode()[0]
    print(col_name, value)
    df_concat[col_name].fillna(value, inplace=True)
    
col_str = 'balance_limit,balance_log,balance_limit_log,balance_account_avg_log,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min' 
for col_name in col_str.split(',') : 
    value = df_concat[col_name].mean(skipna=True)
    df_concat[col_name].fillna(value,inplace=True)
print('isna \n',df_concat.isna().sum())

#TODO 离群点处理

#保存结果
df_concat.to_csv(output_path, index=False)
print('output_path : ', output_path)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
# !ls

(20054, 58)
Index(['id', 'title', 'career', 'zip_code', 'residence', 'loan', 'term',
       'interest_rate', 'issue_time', 'syndicated', 'installment',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'level', 'balance_account_avg',
       'loan_term_avg', 'balance_accounts_ratio', 'loan_log',
       'balance_account_avg_log', 'loan_term_avg_log',
       'balance_accounts_ratio_log', 'interest_rate_log', 'balance_log',
       'balance_limit_log', 'balance_accounts_log', 'zip_province', 'zip_city',
       'level_hash', 'level_ord', 'grade', 'interest_rate_cut',
       'interest_rate_log_cut', 'balance_cut', 'loan_cut', 'balance_limit_cut',
       'loan_term_avg_cut', 'balance_account_avg_cut', 'record_time_year',
       'record_time_month', 'record_time_week', 'record_time_year_month',
       'tx_max_min_days', 'tx_count', 'total_amount', '1_amount', '0_amount',
       'total_amount_avg', '1_amount_avg', '0_amount_avg', 'tota

In [128]:
print('done  time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
!wc -l $output_path
df_concat

done  time :  2025-08-31 17:15:21
20055 test.dat.v21


,id,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,balance_account_avg,loan_term_avg,balance_accounts_ratio,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,zip_province,zip_city,level_hash,level_ord,grade,interest_rate_cut,interest_rate_log_cut,balance_cut,loan_cut,balance_limit_cut,loan_term_avg_cut,balance_account_avg_cut,record_time_year,record_time_month,record_time_week,record_time_year_month,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min
0,53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,0,1054600080,795400642,17,9,10100.0,8233.00,B0,914.777778,165.277778,0.529412,8.691146,6.818681,5.107628,-0.635989,2.646175,9.015906,9.220291,2.197225,51,5120,54,6.0,2,17,27,34,2,28,3,32,2003,6,23,200306,165.000000,46.000000,61815.520000,11245.520000,50570.000000,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826,0.278788,1.053302e+09,1.039046e+09
1,53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,0,1172882338,756003297,17,9,58600.0,23460.00,A0,2606.666667,426.388889,0.529412,9.638871,7.865828,6.055352,-0.635989,1.658228,10.063052,10.978490,2.197225,40,4018,212,0.0,1,0,0,46,15,47,14,47,2007,3,9,200703,144.000000,56.000000,38555.840000,20161.940000,18393.900000,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500,0.388889,1.170806e+09,1.158365e+09
2,53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,0,1277511093,658715225,17,9,15500.0,7853.00,B1,872.555556,306.944444,0.529412,9.310186,6.771426,5.726667,-0.635989,2.613740,8.968651,9.648595,2.197225,32,3216,868,7.0,2,16,26,34,12,32,11,30,2010,6,25,201006,172.000000,74.000000,81589.800000,44977.460000,36612.340000,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351,0.430233,1.275350e+09,1.260490e+09
3,53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,0,1198371199,774316843,17,9,10100.0,5815.00,B2,646.111111,426.388889,0.529412,9.638871,6.470971,6.055352,-0.635989,2.653242,8.668196,9.220291,2.197225,23,2313,222,8.0,2,17,27,31,15,28,14,15,2007,12,51,200712,159.000000,73.000000,48205.320000,24048.520000,24156.800000,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068,0.459119,1.195776e+09,1.182038e+09
4,53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,1,1214957804,832207268,17,9,13100.0,8225.00,A3,913.888889,216.666667,0.529412,8.961879,6.817709,5.378360,-0.635989,2.302585,9.014934,9.480368,2.197225,17,1711,696,3.0,1,9,18,34,5,31,7,32,2008,7,27,200807,174.000000,91.000000,78113.610000,29797.860000,48315.750000,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308,0.522989,1.214438e+09,1.199405e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20049,73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,0,1125964800,1018224000,6,3,3818.0,2224.69,A4,741.563333,833.333333,0.500000,9.210340,6.608761,6.725434,-0.693147,2.936513,7.707373,8.247482,1.098612,60,6011,625,4.0,1,26,35,20,9,21,23,20,2005,9,36,200509,172.017237,79.054523,59098.413419,26663.291556,32435.121863,343.825720,155.300446,188.525274,801.780851,328.960726,472.820124,0.459716,1.176744e+09,1.161882e+09
20050,73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,0,1157068800,1054425600,6,6,5502.0,4126.71,B4,687.785000,833.333333,1.000000,9.210340,6.533476,6.725434,0.000000,3.377588,8.325236,8.612867,1.791759,60,6011,403,10.0,2,46,48,28,9,24,23,17,2006,9,35,200609,172.017237,79.054523,59098.413419,26663.291556,32435.121863,343.825720,155.300446,188.525274,801.780851,328.960726,472.820124,0.459716,1.176744e+09,1.161882e+09
20051,73531,